## Managing Missing Values

## 1. Setting

In [1]:
%AddJar file:///home/jovyan/work/apps/Emiasd-Flight-Data-Analysis.jar

Starting download from file:///home/jovyan/work/apps/Emiasd-Flight-Data-Analysis.jar
Finished download of Emiasd-Flight-Data-Analysis.jar
Using cached version of Emiasd-Flight-Data-Analysis.jar


In [2]:
import org.apache.spark.sql.SparkSession
import com.flightdelay.config.{AppConfiguration, ConfigurationLoader, ExperimentConfig}
import com.flightdelay.data.loaders.FlightDataLoader

// Env Configuration
val args: Array[String] = Array("jupyter")
implicit val configuration: AppConfiguration = ConfigurationLoader.loadConfiguration(args)
implicit val experimentConfig: ExperimentConfig = configuration.experiments(0)

val spark = SparkSession.builder()
  .config(sc.getConf)
  .config("spark.eventLog.enabled", "true")
  .config("spark.eventLog.dir", s"${configuration.common.output.basePath}/spark-events")  // ex: "file:/tmp/spark-events" ou "hdfs:///spark-events"
  .getOrCreate()

// Rendre la session Spark implicite
implicit val session = spark


args = Array(jupyter)
configuration = AppConfiguration(local,CommonConfig(42,true,debug,false,false,DataConfig(/home/jovyan/work/data,FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Flights/201201*.csv),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Weather/20101*.txt),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/wban_airport_timezone.csv)),OutputConfig(/home/jovyan/work/output,FileConfig(/home/jovyan/work/output/data),FileConfig(/home/jovyan/work/output/model)),MLFlowConfig(false,http://localhost:5555)),Stream(ExperimentConfig(Experience-local,Baseline Random ...


AppConfiguration(local,CommonConfig(42,true,debug,false,false,DataConfig(/home/jovyan/work/data,FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Flights/201201*.csv),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Weather/20101*.txt),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/wban_airport_timezone.csv)),OutputConfig(/home/jovyan/work/output,FileConfig(/home/jovyan/work/output/data),FileConfig(/home/jovyan/work/output/model)),MLFlowConfig(false,http://localhost:5555)),Stream(ExperimentConfig(Experience-local,Baseline Random ...

## 2.Data Loading

In [3]:
val flightDFPath = s"${configuration.common.output.basePath}/common/data/raw_flights.parquet"
val flightData = spark.read.parquet(flightDFPath)

val weatherDFPath = s"${configuration.common.output.basePath}/common/data/raw_weather.parquet"
val weatherData = spark.read.parquet(weatherDFPath)

println("Weather DF Count: ", weatherData.count())

println("Flight DF Count: ", flightData.count())

(Weather DF Count: ,32631312)
(Flight DF Count: ,18286055)


flightDFPath = /home/jovyan/work/output/common/data/raw_flights.parquet
flightData = [FL_DATE: date, OP_CARRIER_AIRLINE_ID: int ... 10 more fields]
weatherDFPath = /home/jovyan/work/output/common/data/raw_weather.parquet
weatherData = [WBAN: string, Date: string ... 42 more fields]


[WBAN: string, Date: string ... 42 more fields]

## 3.Flights

### 3.1 Missing Values

In [7]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._

val df = flightData

val missingExprs = df.schema.fields.map { f =>
  val c  = f.name
  val dt = f.dataType
  val colExpr = col(c)

  val cond = dt match {
    case DoubleType | FloatType =>
      colExpr.isNull || isnan(colExpr)

    case StringType =>
      colExpr.isNull || colExpr === ""

    // pour DATE, INT, LONG, etc. → on ne teste que null
    case _ =>
      colExpr.isNull
  }

  sum(when(cond, 1).otherwise(0)).alias(c)
}

val missingDf = df.agg(missingExprs.head, missingExprs.tail: _*)
missingDf.show(20, 1000, true)

-RECORD 0-------------------------
 FL_DATE               | 0        
 OP_CARRIER_AIRLINE_ID | 0        
 OP_CARRIER_FL_NUM     | 0        
 ORIGIN_AIRPORT_ID     | 0        
 DEST_AIRPORT_ID       | 0        
 CRS_DEP_TIME          | 1        
 ARR_DELAY_NEW         | 342986   
 CANCELLED             | 18286055 
 DIVERTED              | 18286055 
 CRS_ELAPSED_TIME      | 12       
 WEATHER_DELAY         | 14761092 
 NAS_DELAY             | 14761092 



df = [FL_DATE: date, OP_CARRIER_AIRLINE_ID: int ... 10 more fields]
missingExprs = Array(sum(CASE WHEN (FL_DATE IS NULL) THEN 1 ELSE 0 END) AS FL_DATE, sum(CASE WHEN (OP_CARRIER_AIRLINE_ID IS NULL) THEN 1 ELSE 0 END) AS OP_CARRIER_AIRLINE_ID, sum(CASE WHEN (OP_CARRIER_FL_NUM IS NULL) THEN 1 ELSE 0 END) AS OP_CARRIER_FL_NUM, sum(CASE WHEN (ORIGIN_AIRPORT_ID IS NULL) THEN 1 ELSE 0 END) AS ORIGIN_AIRPORT_ID, sum(CASE WHEN (DEST_AIRPORT_ID IS NULL) THEN 1 ELSE 0 END) AS DEST_AIRPORT_ID, sum(CASE WHEN (CRS_DEP_TIME IS NULL) THEN 1 ELSE 0 END) AS CRS_DEP_TIME, sum(CASE WHEN ((ARR_DELAY_NEW IS NULL) OR isnan(ARR_DELAY_NEW)) THEN 1 ELSE 0 END) AS ARR_DE...


Array(sum(CASE WHEN (FL_DATE IS NULL) THEN 1 ELSE 0 END) AS FL_DATE, sum(CASE WHEN (OP_CARRIER_AIRLINE_ID IS NULL) THEN 1 ELSE 0 END) AS OP_CARRIER_AIRLINE_ID, sum(CASE WHEN (OP_CARRIER_FL_NUM IS NULL) THEN 1 ELSE 0 END) AS OP_CARRIER_FL_NUM, sum(CASE WHEN (ORIGIN_AIRPORT_ID IS NULL) THEN 1 ELSE 0 END) AS ORIGIN_AIRPORT_ID, sum(CASE WHEN (DEST_AIRPORT_ID IS NULL) THEN 1 ELSE 0 END) AS DEST_AIRPORT_ID, sum(CASE WHEN (CRS_DEP_TIME IS NULL) THEN 1 ELSE 0 END) AS CRS_DEP_TIME, sum(CASE WHEN ((ARR_DELAY_NEW IS NULL) OR isnan(ARR_DELAY_NEW)) THEN 1 ELSE 0 END) AS ARR_DE...

In [8]:
flightData.printSchema

root
 |-- FL_DATE: date (nullable = true)
 |-- OP_CARRIER_AIRLINE_ID: integer (nullable = true)
 |-- OP_CARRIER_FL_NUM: integer (nullable = true)
 |-- ORIGIN_AIRPORT_ID: integer (nullable = true)
 |-- DEST_AIRPORT_ID: integer (nullable = true)
 |-- CRS_DEP_TIME: integer (nullable = true)
 |-- ARR_DELAY_NEW: double (nullable = true)
 |-- CANCELLED: integer (nullable = true)
 |-- DIVERTED: integer (nullable = true)
 |-- CRS_ELAPSED_TIME: double (nullable = true)
 |-- WEATHER_DELAY: double (nullable = true)
 |-- NAS_DELAY: double (nullable = true)



In [9]:
flightData.show(1, 1000, true)

-RECORD 0---------------------------
 FL_DATE               | 2013-07-01 
 OP_CARRIER_AIRLINE_ID | 20363      
 OP_CARRIER_FL_NUM     | 3407       
 ORIGIN_AIRPORT_ID     | 11433      
 DEST_AIRPORT_ID       | 13342      
 CRS_DEP_TIME          | 1040       
 ARR_DELAY_NEW         | 0.0        
 CANCELLED             | NULL       
 DIVERTED              | NULL       
 CRS_ELAPSED_TIME      | 79.0       
 WEATHER_DELAY         | NULL       
 NAS_DELAY             | NULL       
only showing top 1 row



### 3.1 Check Formatting

In [13]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._

val checks = flightData.schema.fields.map { f =>
  val c = f.name
  val dt = f.dataType
  val colExpr = col(c)

  val check = dt match {

    // ---------- DATE ----------
    case DateType =>
      sum( when(colExpr.isNull, 1).otherwise(0) ).alias(s"${c}_invalid")

    // ---------- INTEGER ----------
    case IntegerType =>
      sum(
        when(colExpr.isNull, 1)
          .when(colExpr.cast(IntegerType).isNull, 1)
          .otherwise(0)
      ).alias(s"${c}_invalid")

    // ---------- DOUBLE ----------
    case DoubleType =>
      sum(
        when(colExpr.isNull, 1)
          .when(isnan(colExpr), 1)
          .otherwise(0)
      ).alias(s"${c}_invalid")

    // For safety
    case _ =>
      sum(when(colExpr.isNull, 1).otherwise(0)).alias(s"${c}_invalid")
  }

  check
}

// Agrégation finale
val validationDF = flightData.agg(checks.head, checks.tail: _*)

validationDF.show(20, 1000, true)

-RECORD 0---------------------------------
 FL_DATE_invalid               | 0        
 OP_CARRIER_AIRLINE_ID_invalid | 0        
 OP_CARRIER_FL_NUM_invalid     | 0        
 ORIGIN_AIRPORT_ID_invalid     | 0        
 DEST_AIRPORT_ID_invalid       | 0        
 CRS_DEP_TIME_invalid          | 1        
 ARR_DELAY_NEW_invalid         | 342986   
 CANCELLED_invalid             | 18286055 
 DIVERTED_invalid              | 18286055 
 CRS_ELAPSED_TIME_invalid      | 12       
 WEATHER_DELAY_invalid         | 14761092 
 NAS_DELAY_invalid             | 14761092 



checks = Array(sum(CASE WHEN (FL_DATE IS NULL) THEN 1 ELSE 0 END) AS FL_DATE_invalid, sum(CASE WHEN (OP_CARRIER_AIRLINE_ID IS NULL) THEN 1 WHEN (CAST(OP_CARRIER_AIRLINE_ID AS INT) IS NULL) THEN 1 ELSE 0 END) AS OP_CARRIER_AIRLINE_ID_invalid, sum(CASE WHEN (OP_CARRIER_FL_NUM IS NULL) THEN 1 WHEN (CAST(OP_CARRIER_FL_NUM AS INT) IS NULL) THEN 1 ELSE 0 END) AS OP_CARRIER_FL_NUM_invalid, sum(CASE WHEN (ORIGIN_AIRPORT_ID IS NULL) THEN 1 WHEN (CAST(ORIGIN_AIRPORT_ID AS INT) IS NULL) THEN 1 ELSE 0 END) AS ORIGIN_AIRPORT_ID_invalid, sum(CASE WHEN (DEST_AIRPORT_ID IS NULL) THEN 1 WHEN (CAST(DEST_AIRPORT_ID AS INT) IS NULL) THEN 1 ELSE 0 END) AS DEST_AIRPORT_ID_invalid, sum(CASE WHEN (CR...


Array(sum(CASE WHEN (FL_DATE IS NULL) THEN 1 ELSE 0 END) AS FL_DATE_invalid, sum(CASE WHEN (OP_CARRIER_AIRLINE_ID IS NULL) THEN 1 WHEN (CAST(OP_CARRIER_AIRLINE_ID AS INT) IS NULL) THEN 1 ELSE 0 END) AS OP_CARRIER_AIRLINE_ID_invalid, sum(CASE WHEN (OP_CARRIER_FL_NUM IS NULL) THEN 1 WHEN (CAST(OP_CARRIER_FL_NUM AS INT) IS NULL) THEN 1 ELSE 0 END) AS OP_CARRIER_FL_NUM_invalid, sum(CASE WHEN (ORIGIN_AIRPORT_ID IS NULL) THEN 1 WHEN (CAST(ORIGIN_AIRPORT_ID AS INT) IS NULL) THEN 1 ELSE 0 END) AS ORIGIN_AIRPORT_ID_invalid, sum(CASE WHEN (DEST_AIRPORT_ID IS NULL) THEN 1 WHEN (CAST(DEST_AIRPORT_ID AS INT) IS NULL) THEN 1 ELSE 0 END) AS DEST_AIRPORT_ID_invalid, sum(CASE WHEN (CR...

In [15]:
import org.apache.spark.sql.functions._

// Règle stricte HHMM
val invalidCRS = flightData.filter(
  col("CRS_DEP_TIME").isNull ||
  col("CRS_DEP_TIME").cast("int").isNull ||
  !(col("CRS_DEP_TIME").between(0, 2359)) ||
  (col("CRS_DEP_TIME") % 100 >= 60)
)

invalidCRS.show(1, 1000, true)

-RECORD 0---------------------------
 FL_DATE               | 2012-07-12 
 OP_CARRIER_AIRLINE_ID | 19393      
 OP_CARRIER_FL_NUM     | 935        
 ORIGIN_AIRPORT_ID     | 10423      
 DEST_AIRPORT_ID       | 12191      
 CRS_DEP_TIME          | NULL       
 ARR_DELAY_NEW         | 612.0      
 CANCELLED             | NULL       
 DIVERTED              | NULL       
 CRS_ELAPSED_TIME      | NULL       
 WEATHER_DELAY         | 0.0        
 NAS_DELAY             | 64.0       



invalidCRS = [FL_DATE: date, OP_CARRIER_AIRLINE_ID: int ... 10 more fields]


[FL_DATE: date, OP_CARRIER_AIRLINE_ID: int ... 10 more fields]

In [33]:
flightData.filter(col("CRS_ELAPSED_TIME").isNull).count()

res29: Long = 12


In [34]:
flightData.filter(col("CRS_DEP_TIME").isNull).count()

res30: Long = 1


In [36]:
flightData
  .filter(col("CRS_ELAPSED_TIME").isNull)
  .select(
    "CRS_DEP_TIME",
    "ARR_DELAY_NEW",
    "CANCELLED",
    "DIVERTED",
    "CRS_ELAPSED_TIME",
    "WEATHER_DELAY",
    "NAS_DELAY"
  )
  .show(200, false)

+------------+-------------+---------+--------+----------------+-------------+---------+
|CRS_DEP_TIME|ARR_DELAY_NEW|CANCELLED|DIVERTED|CRS_ELAPSED_TIME|WEATHER_DELAY|NAS_DELAY|
+------------+-------------+---------+--------+----------------+-------------+---------+
|NULL        |612.0        |NULL     |NULL    |NULL            |0.0          |64.0     |
|830         |NULL         |NULL     |NULL    |NULL            |NULL         |NULL     |
|1225        |NULL         |NULL     |NULL    |NULL            |NULL         |NULL     |
|2055        |NULL         |NULL     |NULL    |NULL            |NULL         |NULL     |
|1500        |NULL         |NULL     |NULL    |NULL            |NULL         |NULL     |
|1022        |NULL         |NULL     |NULL    |NULL            |NULL         |NULL     |
|1428        |NULL         |NULL     |NULL    |NULL            |NULL         |NULL     |
|1605        |NULL         |NULL     |NULL    |NULL            |NULL         |NULL     |
|1933        |NULL   

In [38]:
flightData
  .filter(col("CRS_DEP_TIME").isNull)
  .show(1, 200, true)

-RECORD 0---------------------------
 FL_DATE               | 2012-07-12 
 OP_CARRIER_AIRLINE_ID | 19393      
 OP_CARRIER_FL_NUM     | 935        
 ORIGIN_AIRPORT_ID     | 10423      
 DEST_AIRPORT_ID       | 12191      
 CRS_DEP_TIME          | NULL       
 ARR_DELAY_NEW         | 612.0      
 CANCELLED             | NULL       
 DIVERTED              | NULL       
 CRS_ELAPSED_TIME      | NULL       
 WEATHER_DELAY         | 0.0        
 NAS_DELAY             | 64.0       



## 3.4 Check Values

In [30]:
flightData.printSchema

lastException: Throwable = null


root
 |-- FL_DATE: date (nullable = true)
 |-- OP_CARRIER_AIRLINE_ID: integer (nullable = true)
 |-- OP_CARRIER_FL_NUM: integer (nullable = true)
 |-- ORIGIN_AIRPORT_ID: integer (nullable = true)
 |-- DEST_AIRPORT_ID: integer (nullable = true)
 |-- CRS_DEP_TIME: integer (nullable = true)
 |-- ARR_DELAY_NEW: double (nullable = true)
 |-- CANCELLED: integer (nullable = true)
 |-- DIVERTED: integer (nullable = true)
 |-- CRS_ELAPSED_TIME: double (nullable = true)
 |-- WEATHER_DELAY: double (nullable = true)
 |-- NAS_DELAY: double (nullable = true)



In [31]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.Column
import org.apache.spark.sql.types._

// Fonction utilitaire : détecte si une valeur est numérique (regex sur string)
def isNumericCol(c: Column): Column =
  c.cast("string").rlike("^[+-]?\\d+(\\.\\d+)?$")

println("===== Parcours colonne par colonne =====\n")

flightData.columns.foreach { c =>

  println(s"\n========================")
  println(s"Colonne : $c")
  println("========================")

  val raw     = col(c).cast("string")
  val trimmed = trim(raw)

  // Colonne d’affichage : <NULL>, <SPACES> ou valeur d’origine
  val displayCol =
    when(col(c).isNull, lit("<NULL>"))
      .when(trimmed === "", lit("<SPACES>"))
      .otherwise(raw)
      .alias(c)

  // On garde aussi un flag "est numérique ?" pour l'orderBy
  val ordered = flightData
    .select(
      displayCol,
      isNumericCol(trimmed).alias("_isNum")
    )
    .distinct()
    .orderBy(
      col("_isNum").asc,   // false (non numérique) → en premier
      col(c).asc           // puis tri alphabétique / numérique sur la valeur affichée
    )
    .select(col(c))        // on ne garde que la colonne affichée
    .limit(10)

  ordered.show(false)
}

===== Parcours colonne par colonne =====


Colonne : FL_DATE
+----------+
|FL_DATE   |
+----------+
|2012-01-01|
|2012-01-02|
|2012-01-03|
|2012-01-04|
|2012-01-05|
|2012-01-06|
|2012-01-07|
|2012-01-08|
|2012-01-09|
|2012-01-10|
+----------+


Colonne : OP_CARRIER_AIRLINE_ID
+---------------------+
|OP_CARRIER_AIRLINE_ID|
+---------------------+
|19393                |
|19690                |
|19790                |
|19805                |
|19930                |
|19977                |
|20304                |
|20355                |
|20363                |
|20366                |
+---------------------+


Colonne : OP_CARRIER_FL_NUM
+-----------------+
|OP_CARRIER_FL_NUM|
+-----------------+
|1                |
|10               |
|100              |
|1000             |
|1001             |
|1002             |
|1003             |
|1004             |
|1005             |
|1006             |
+-----------------+


Colonne : ORIGIN_AIRPORT_ID
+-----------------+
|ORIGIN_AIRPORT_ID|
+------

isNumericCol: (c: org.apache.spark.sql.Column)org.apache.spark.sql.Column


+---------+
|NAS_DELAY|
+---------+
|<NULL>   |
|0.0      |
|1.0      |
|10.0     |
|100.0    |
|1007.0   |
|101.0    |
|1017.0   |
|102.0    |
|103.0    |
+---------+



## 4. Weather

### 4.1 Missing Values

In [17]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._

val df = weatherData

val missingExprs = df.schema.fields.map { f =>
  val c  = f.name
  val dt = f.dataType
  val colExpr = col(c)

  val cond = dt match {
    case DoubleType | FloatType =>
      colExpr.isNull || isnan(colExpr)

    case StringType =>
      colExpr.isNull || colExpr === ""

    // pour DATE, INT, LONG, etc. → on ne teste que null
    case _ =>
      colExpr.isNull
  }

  sum(when(cond, 1).otherwise(0)).alias(c)
}

df = [WBAN: string, Date: string ... 42 more fields]
missingExprs = Array(sum(CASE WHEN ((WBAN IS NULL) OR (WBAN = )) THEN 1 ELSE 0 END) AS WBAN, sum(CASE WHEN ((Date IS NULL) OR (Date = )) THEN 1 ELSE 0 END) AS Date, sum(CASE WHEN ((Time IS NULL) OR (Time = )) THEN 1 ELSE 0 END) AS Time, sum(CASE WHEN (StationType IS NULL) THEN 1 ELSE 0 END) AS StationType, sum(CASE WHEN ((SkyCondition IS NULL) OR (SkyCondition = )) THEN 1 ELSE 0 END) AS SkyCondition, sum(CASE WHEN ((SkyConditionFlag IS NULL) OR (SkyConditionFlag = )) THEN 1 ELSE 0 END) AS SkyConditionFlag, sum(CASE WHEN ((Visibility IS NULL) OR isnan(Visibility)) THEN 1 ELSE 0 END) AS Visibili...


Array(sum(CASE WHEN ((WBAN IS NULL) OR (WBAN = )) THEN 1 ELSE 0 END) AS WBAN, sum(CASE WHEN ((Date IS NULL) OR (Date = )) THEN 1 ELSE 0 END) AS Date, sum(CASE WHEN ((Time IS NULL) OR (Time = )) THEN 1 ELSE 0 END) AS Time, sum(CASE WHEN (StationType IS NULL) THEN 1 ELSE 0 END) AS StationType, sum(CASE WHEN ((SkyCondition IS NULL) OR (SkyCondition = )) THEN 1 ELSE 0 END) AS SkyCondition, sum(CASE WHEN ((SkyConditionFlag IS NULL) OR (SkyConditionFlag = )) THEN 1 ELSE 0 END) AS SkyConditionFlag, sum(CASE WHEN ((Visibility IS NULL) OR isnan(Visibility)) THEN 1 ELSE 0 END) AS Visibili...

In [18]:
val missingDf = df.agg(missingExprs.head, missingExprs.tail: _*)
missingDf.show(100, 1000, true)

-RECORD 0-----------------------------
 WBAN                      | 0        
 Date                      | 0        
 Time                      | 0        
 StationType               | 0        
 SkyCondition              | 0        
 SkyConditionFlag          | 0        
 Visibility                | 14174863 
 VisibilityFlag            | 0        
 WeatherType               | 0        
 WeatherTypeFlag           | 0        
 DryBulbFarenheit          | 156168   
 DryBulbFarenheitFlag      | 0        
 DryBulbCelsius            | 156168   
 DryBulbCelsiusFlag        | 0        
 WetBulbFarenheit          | 14631473 
 WetBulbFarenheitFlag      | 0        
 WetBulbCelsius            | 14631473 
 WetBulbCelsiusFlag        | 0        
 DewPointFarenheit         | 14059176 
 DewPointFarenheitFlag     | 0        
 DewPointCelsius           | 14059176 
 DewPointCelsiusFlag       | 0        
 RelativeHumidity          | 14631473 
 RelativeHumidityFlag      | 0        
 WindSpeed               

missingDf = [WBAN: bigint, Date: bigint ... 42 more fields]


[WBAN: bigint, Date: bigint ... 42 more fields]

In [21]:
weatherData.printSchema

root
 |-- WBAN: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- StationType: integer (nullable = true)
 |-- SkyCondition: string (nullable = true)
 |-- SkyConditionFlag: string (nullable = true)
 |-- Visibility: double (nullable = true)
 |-- VisibilityFlag: string (nullable = true)
 |-- WeatherType: string (nullable = true)
 |-- WeatherTypeFlag: string (nullable = true)
 |-- DryBulbFarenheit: double (nullable = true)
 |-- DryBulbFarenheitFlag: string (nullable = true)
 |-- DryBulbCelsius: double (nullable = true)
 |-- DryBulbCelsiusFlag: string (nullable = true)
 |-- WetBulbFarenheit: double (nullable = true)
 |-- WetBulbFarenheitFlag: string (nullable = true)
 |-- WetBulbCelsius: double (nullable = true)
 |-- WetBulbCelsiusFlag: string (nullable = true)
 |-- DewPointFarenheit: double (nullable = true)
 |-- DewPointFarenheitFlag: string (nullable = true)
 |-- DewPointCelsius: double (nullable = true)
 |-- DewPointCelsiusFlag: string

In [20]:
print(weatherData.count())

32631312

### 4.2 Suspect values

In [22]:
import org.apache.spark.sql.functions._

// 1) Sélection des colonnes Flag
val flagCols = weatherData.columns.filter(_.endsWith("Flag"))

// 2) Construction des expressions de count
val countExprs = flagCols.map { c =>
  sum( when(col(c) === "s", 1).otherwise(0) ).alias(c)
}

// 3) Agrégation en un seul DataFrame
val result = weatherData.agg(countExprs.head, countExprs.tail: _*)

// 4) Affichage
result.show(100, 1000, true)

-RECORD 0--------------------------
 SkyConditionFlag          | 29753 
 VisibilityFlag            | 30141 
 WeatherTypeFlag           | 26319 
 DryBulbFarenheitFlag      | 0     
 DryBulbCelsiusFlag        | 61261 
 WetBulbFarenheitFlag      | 0     
 WetBulbCelsiusFlag        | 0     
 DewPointFarenheitFlag     | 0     
 DewPointCelsiusFlag       | 33228 
 RelativeHumidityFlag      | 0     
 WindSpeedFlag             | 5953  
 WindDirectionFlag         | 1     
 ValueForWindCharacterFlag | 12099 
 StationPressureFlag       | 0     
 PressureTendencyFlag      | 0     
 PressureChangeFlag        | 0     
 SeaLevelPressureFlag      | 1348  
 RecordTypeFlag            | 0     
 HourlyPrecipFlag          | 8712  
 AltimeterFlag             | 1262  



flagCols = Array(SkyConditionFlag, VisibilityFlag, WeatherTypeFlag, DryBulbFarenheitFlag, DryBulbCelsiusFlag, WetBulbFarenheitFlag, WetBulbCelsiusFlag, DewPointFarenheitFlag, DewPointCelsiusFlag, RelativeHumidityFlag, WindSpeedFlag, WindDirectionFlag, ValueForWindCharacterFlag, StationPressureFlag, PressureTendencyFlag, PressureChangeFlag, SeaLevelPressureFlag, RecordTypeFlag, HourlyPrecipFlag, AltimeterFlag)
countExprs = Array(sum(CASE WHEN (SkyConditionFlag = s) THEN 1 ELSE 0 END) AS SkyConditionFlag, sum(CASE WHEN (VisibilityFlag = s) THEN 1 ELSE 0 END) AS VisibilityFlag, sum(CASE WHEN (WeatherTypeFlag = s) THEN 1 ELSE 0 END) AS WeatherTypeFlag, sum(CASE WHEN (DryBulbFarenheitFlag = s) THEN 1 E...


Array(sum(CASE WHEN (SkyConditionFlag = s) THEN 1 ELSE 0 END) AS SkyConditionFlag, sum(CASE WHEN (VisibilityFlag = s) THEN 1 ELSE 0 END) AS VisibilityFlag, sum(CASE WHEN (WeatherTypeFlag = s) THEN 1 ELSE 0 END) AS WeatherTypeFlag, sum(CASE WHEN (DryBulbFarenheitFlag = s) THEN 1 E...

### 4.3 Numeric Columns with Chars

In [24]:
import org.apache.spark.sql.functions._

val df = weatherData

// Colonnes numériques attendues
val numericCols = Seq(
  "Visibility",
  "DryBulbFarenheit", "DryBulbCelsius",
  "WetBulbFarenheit", "WetBulbCelsius",
  "DewPointFarenheit", "DewPointCelsius",
  "RelativeHumidity",
  "WindSpeed",
  "WindDirection",
  "StationPressure",
  "PressureChange",
  "Altimeter"
)

// Condition générique pour détecter une valeur non-numérique
def badCond(c: String) =
  trim(col(c)) === "" ||                      // vide / blancs
  col(c).cast("double").isNull               // cast impossible = non-numérique

// --- 1) COUNT des valeurs non numériques ---
val countExprs = numericCols.map { c =>
  sum( when(badCond(c), 1).otherwise(0) ).alias(c)
}

val countsDf = df.agg(countExprs.head, countExprs.tail: _*)
countsDf.show(100, 1000, true)


// --- 2) Valeurs fautives distinctes pour chaque colonne ---
numericCols.foreach { c =>
  println("\n==============================")
  println(s"Colonne : $c — valeurs distinctes NON numériques")
  println("==============================")

  df.filter(badCond(c))
    .select(trim(col(c)).as(c))
    .distinct()
    .show(100, 1000, true)
}

-RECORD 0---------------------
 Visibility        | 14174863 
 DryBulbFarenheit  | 156168   
 DryBulbCelsius    | 156168   
 WetBulbFarenheit  | 14631473 
 WetBulbCelsius    | 14631473 
 DewPointFarenheit | 14059176 
 DewPointCelsius   | 14059176 
 RelativeHumidity  | 14631473 
 WindSpeed         | 12988107 
 WindDirection     | 14070730 
 StationPressure   | 14439520 
 PressureChange    | 31168523 
 Altimeter         | 13929988 


Colonne : Visibility — valeurs distinctes NON numériques
-RECORD 0----------
 Visibility | NULL 


Colonne : DryBulbFarenheit — valeurs distinctes NON numériques
-RECORD 0----------------
 DryBulbFarenheit | NULL 


Colonne : DryBulbCelsius — valeurs distinctes NON numériques
-RECORD 0--------------
 DryBulbCelsius | NULL 


Colonne : WetBulbFarenheit — valeurs distinctes NON numériques
-RECORD 0----------------
 WetBulbFarenheit | NULL 


Colonne : WetBulbCelsius — valeurs distinctes NON numériques
-RECORD 0--------------
 WetBulbCelsius | NULL 


Colonne :

df = [WBAN: string, Date: string ... 42 more fields]
numericCols = List(Visibility, DryBulbFarenheit, DryBulbCelsius, WetBulbFarenheit, WetBulbCelsius, DewPointFarenheit, DewPointCelsius, RelativeHumidity, WindSpeed, WindDirection, StationPressure, PressureChange, Altimeter)
countExprs = List(sum(CASE WHEN ((trim(Visibility) = ) OR (CAST(Visibility AS DOUBLE) IS NULL)) THEN 1 ELSE 0 END) AS Visibility, sum(CASE WHEN ((trim(DryBulbFarenheit) = ) OR (CAST(DryBulbFarenheit AS DOUBLE) IS NULL)) THEN 1 ELSE 0 END) AS DryBulbFarenheit, sum(CASE WHEN ((trim(DryBulbCelsius) = ) OR (CAST(DryBulbCelsius AS DOUBLE) IS NULL)) THEN 1 E...


badCond: (c: String)org.apache.spark.sql.Column


List(sum(CASE WHEN ((trim(Visibility) = ) OR (CAST(Visibility AS DOUBLE) IS NULL)) THEN 1 ELSE 0 END) AS Visibility, sum(CASE WHEN ((trim(DryBulbFarenheit) = ) OR (CAST(DryBulbFarenheit AS DOUBLE) IS NULL)) THEN 1 ELSE 0 END) AS DryBulbFarenheit, sum(CASE WHEN ((trim(DryBulbCelsius) = ) OR (CAST(DryBulbCelsius AS DOUBLE) IS NULL)) THEN 1 E...

### Analyse Numeric Vs Non Numeric

In [26]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import org.apache.spark.sql._

// =========================================================================
// 1) Récupérer toutes les colonnes de type String
// =========================================================================
val df = weatherData   // adapte si ton DataFrame a un autre nom

val stringCols: Seq[String] =
  df.schema.fields.collect {
    case StructField(name, StringType, _, _) => name
  }

// =========================================================================
// 2) Fonction de profilage d'une colonne string
//    - détecte les valeurs "numériques"
//    - compte M, s, null, vides, etc.
// =========================================================================

// Regex pour "nombre" (entier ou décimal, signe optionnel)
def numericCond(c: Column): Column =
  c.rlike("^[+-]?\\d+(\\.\\d+)?$")

case class StringColProfile(
  colName: String,
  total: Long,
  nNull: Long,
  nEmpty: Long,
  nNumericLike: Long,
  nNonNumericNonEmpty: Long,
  nM: Long,
  nS: Long,
  distinctNonNumeric: Seq[String],
  numericRatio: Double
)

val profiles: Seq[StringColProfile] = stringCols.map { colName =>
  val cTrim = trim(col(colName))

  val aggDf = df.agg(
    count(lit(1)).alias("total"),
    sum(when(col(colName).isNull, 1).otherwise(0)).alias("nNull"),
    sum(when(cTrim === "", 1).otherwise(0)).alias("nEmpty"),
    sum(when(numericCond(cTrim), 1).otherwise(0)).alias("nNumericLike"),
    sum(
      when(
        !numericCond(cTrim) &&
        !col(colName).isNull &&
        cTrim =!= "",
        1
      ).otherwise(0)
    ).alias("nNonNumericNonEmpty"),
    sum(when(upper(cTrim) === "M", 1).otherwise(0)).alias("nM"),
    sum(when(lower(cTrim) === "s", 1).otherwise(0)).alias("nS")
  )

  val row = aggDf.first()

  val total       = row.getAs[Long]("total")
  val nNull       = row.getAs[Long]("nNull")
  val nEmpty      = row.getAs[Long]("nEmpty")
  val nNumeric    = row.getAs[Long]("nNumericLike")
  val nNonNumNE   = row.getAs[Long]("nNonNumericNonEmpty")
  val nM          = row.getAs[Long]("nM")
  val nS          = row.getAs[Long]("nS")

  val base = total - nNull - nEmpty
  val ratio =
    if (base > 0) nNumeric.toDouble / base.toDouble
    else 0.0

  // Valeurs distinctes non numériques (hors null/vides)
  val distinctNonNum =
    df.filter(
        !numericCond(cTrim) &&
        !col(colName).isNull &&
        cTrim =!= ""
      )
      .select(cTrim.alias(colName))
      .distinct()
      .collect()
      .map(_.getString(0))
      .toSeq

  StringColProfile(
    colName           = colName,
    total             = total,
    nNull             = nNull,
    nEmpty            = nEmpty,
    nNumericLike      = nNumeric,
    nNonNumericNonEmpty = nNonNumNE,
    nM                = nM,
    nS                = nS,
    distinctNonNumeric = distinctNonNum,
    numericRatio      = ratio
  )
}

// =========================================================================
// 3) Affichage des colonnes "candidates numériques"
//    (par ex. ratio >= 0.9 → 90 % des valeurs utiles sont des nombres)
// =========================================================================

val threshold = 0.9

println(s"\n===== Colonnes STRING candidates pour cast en numérique (ratio >= $threshold) =====\n")

profiles
  .filter(_.numericRatio >= threshold)
  .sortBy(- _.numericRatio)
  .foreach { p =>
    println(s"Colonne : ${p.colName}")
    println(f"  total                : ${p.total}%d")
    println(f"  nulls                : ${p.nNull}%d")
    println(f"  vides                : ${p.nEmpty}%d")
    println(f"  numériques-like      : ${p.nNumericLike}%d")
    println(f"  non-numériques (≠'',≠NULL) : ${p.nNonNumericNonEmpty}%d")
    println(f"  'M'                  : ${p.nM}%d")
    println(f"  's'                  : ${p.nS}%d")
    println(f"  ratio numériques     : ${p.numericRatio}%.4f")

    println("  Valeurs NON numériques distinctes :")
    p.distinctNonNumeric.take(50).foreach(v => println(s"    - '$v'"))
    if (p.distinctNonNumeric.size > 50) println("    ... (tronqué)")
    println()
  }


===== Colonnes STRING candidates pour cast en numérique (ratio >= 0.9) =====

Colonne : WBAN
  total                : 32631312
  nulls                : 0
  vides                : 0
  numériques-like      : 32631312
  non-numériques (≠'',≠NULL) : 0
  'M'                  : 0
  's'                  : 0
  ratio numériques     : 1.0000
  Valeurs NON numériques distinctes :

Colonne : Date
  total                : 32631312
  nulls                : 0
  vides                : 0
  numériques-like      : 32631312
  non-numériques (≠'',≠NULL) : 0
  'M'                  : 0
  's'                  : 0
  ratio numériques     : 1.0000
  Valeurs NON numériques distinctes :

Colonne : Time
  total                : 32631312
  nulls                : 0
  vides                : 0
  numériques-like      : 32631312
  non-numériques (≠'',≠NULL) : 0
  'M'                  : 0
  's'                  : 0
  ratio numériques     : 1.0000
  Valeurs NON numériques distinctes :

Colonne : ValueForWindCharacter
  to

df = [WBAN: string, Date: string ... 42 more fields]
stringCols = ArraySeq(WBAN, Date, Time, SkyCondition, SkyConditionFlag, VisibilityFlag, WeatherType, WeatherTypeFlag, DryBulbFarenheitFlag, DryBulbCelsiusFlag, WetBulbFarenheitFlag, WetBulbCelsiusFlag, DewPointFarenheitFlag, DewPointCelsiusFlag, RelativeHumidityFlag, WindSpeedFlag, WindDirectionFlag, ValueForWindCharacter, ValueForWindCharacterFlag, StationPressureFlag, PressureTendency, PressureTendencyFlag, PressureChangeFlag, SeaLevelPressure, SeaLevelPressureFlag, RecordType, RecordTypeFlag, HourlyPrecip, HourlyPrecipFlag, AltimeterFlag)


numericCond: (c: org.apache.spark.sql.Column)...


ArraySeq(WBAN, Date, Time, SkyCondition, SkyConditionFlag, VisibilityFlag, WeatherType, WeatherTypeFlag, DryBulbFarenheitFlag, DryBulbCelsiusFlag, WetBulbFarenheitFlag, WetBulbCelsiusFlag, DewPointFarenheitFlag, DewPointCelsiusFlag, RelativeHumidityFlag, WindSpeedFlag, WindDirectionFlag, ValueForWindCharacter, ValueForWindCharacterFlag, StationPressureFlag, PressureTendency, PressureTendencyFlag, PressureChangeFlag, SeaLevelPressure, SeaLevelPressureFlag, RecordType, RecordTypeFlag, HourlyPrecip, HourlyPrecipFlag, AltimeterFlag)

In [28]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.Column
import org.apache.spark.sql.types._

// Fonction utilitaire : détecte si une valeur est numérique (regex sur string)
def isNumericCol(c: Column): Column =
  c.cast("string").rlike("^[+-]?\\d+(\\.\\d+)?$")

println("===== Parcours colonne par colonne =====\n")

weatherData.columns.foreach { c =>

  println(s"\n========================")
  println(s"Colonne : $c")
  println("========================")

  val raw     = col(c).cast("string")
  val trimmed = trim(raw)

  // Colonne d’affichage : <NULL>, <SPACES> ou valeur d’origine
  val displayCol =
    when(col(c).isNull, lit("<NULL>"))
      .when(trimmed === "", lit("<SPACES>"))
      .otherwise(raw)
      .alias(c)

  // On garde aussi un flag "est numérique ?" pour l'orderBy
  val ordered = weatherData
    .select(
      displayCol,
      isNumericCol(trimmed).alias("_isNum")
    )
    .distinct()
    .orderBy(
      col("_isNum").asc,   // false (non numérique) → en premier
      col(c).asc           // puis tri alphabétique / numérique sur la valeur affichée
    )
    .select(col(c))        // on ne garde que la colonne affichée
    .limit(10)

  ordered.show(false)
}

===== Parcours colonne par colonne =====


Colonne : WBAN
+-----+
|WBAN |
+-----+
|03011|
|03012|
|03013|
|03014|
|03016|
|03017|
|03024|
|03026|
|03027|
|03028|
+-----+


Colonne : Date
+--------+
|Date    |
+--------+
|20120101|
|20120102|
|20120103|
|20120104|
|20120105|
|20120106|
|20120107|
|20120108|
|20120109|
|20120110|
+--------+


Colonne : Time
+----+
|Time|
+----+
|0000|
|0001|
|0002|
|0003|
|0004|
|0005|
|0006|
|0007|
|0008|
|0009|
+----+


Colonne : StationType
+-----------+
|StationType|
+-----------+
|0          |
|11         |
|12         |
|15         |
|4          |
|5          |
|6          |
|9          |
+-----------+


Colonne : SkyCondition
+---------------+
|SkyCondition   |
+---------------+
|BKN            |
|BKN BKN        |
|BKN BKN OVC    |
|BKN OVC        |
|BKN OVCCB      |
|BKN OVCTCU     |
|BKN000 OVC008CB|
|BKN000 OVC015CB|
|BKN001         |
|BKN001 BKN002  |
+---------------+


Colonne : SkyConditionFlag
+----------------+
|SkyConditionFlag|
+-------

isNumericCol: (c: org.apache.spark.sql.Column)org.apache.spark.sql.Column


+-------------+
|AltimeterFlag|
+-------------+
|<SPACES>     |
|s            |
+-------------+



In [39]:
// Map Code → Weather_Type
val typeMap: Map[Int, String] = Map(
  21 -> "Rain", 60 -> "Rain", 61 -> "Rain", 62 -> "Rain", 63 -> "Rain", 64 -> "Rain", 65 -> "Rain",
  24 -> "Freezing_Precip", 56 -> "Freezing_Precip", 57 -> "Freezing_Precip", 66 -> "Freezing_Precip", 67 -> "Freezing_Precip",
  20 -> "Drizzle", 50 -> "Drizzle", 51 -> "Drizzle", 52 -> "Drizzle", 53 -> "Drizzle", 54 -> "Drizzle", 55 -> "Drizzle",
  22 -> "Snow", 70 -> "Snow", 71 -> "Snow", 72 -> "Snow", 73 -> "Snow", 74 -> "Snow", 75 -> "Snow",
  23 -> "Rain_Snow_Mix", 68 -> "Rain_Snow_Mix", 69 -> "Rain_Snow_Mix",
  25 -> "Shower", 26 -> "Shower", 80 -> "Shower", 81 -> "Shower", 82 -> "Shower",
  83 -> "Shower", 84 -> "Shower", 85 -> "Shower", 86 -> "Shower",
  17 -> "Thunderstorm", 29 -> "Thunderstorm", 95 -> "Thunderstorm",
  96 -> "Thunderstorm", 97 -> "Thunderstorm", 99 -> "Thunderstorm",
  10 -> "Fog_Mist", 28 -> "Fog_Mist", 40 -> "Fog_Mist", 41 -> "Fog_Mist",
  42 -> "Fog_Mist", 43 -> "Fog_Mist", 44 -> "Fog_Mist", 45 -> "Fog_Mist",
  5  -> "Haze",
  18 -> "Squall",
  30 -> "Dust_Sand", 31 -> "Dust_Sand", 32 -> "Dust_Sand", 33 -> "Dust_Sand",
  34 -> "Dust_Sand", 35 -> "Dust_Sand",
  36 -> "Blowing_Snow", 37 -> "Blowing_Snow", 38 -> "Blowing_Snow", 39 -> "Blowing_Snow"
)

// Map Code → Weather_Intensity
val intensityMap: Map[Int, String] = Map(
  50 -> "Light", 51 -> "Light", 60 -> "Light", 61 -> "Light", 70 -> "Light", 71 -> "Light",
  80 -> "Light", 83 -> "Light", 85 -> "Light",
  52 -> "Moderate", 53 -> "Moderate", 62 -> "Moderate", 63 -> "Moderate",
  72 -> "Moderate", 73 -> "Moderate", 81 -> "Moderate", 84 -> "Moderate", 86 -> "Moderate",
  54 -> "Heavy", 55 -> "Heavy", 64 -> "Heavy", 65 -> "Heavy",
  74 -> "Heavy", 75 -> "Heavy", 82 -> "Heavy", 87 -> "Heavy", 88 -> "Heavy",
  97 -> "Heavy", 99 -> "Heavy"
)

import org.apache.spark.sql.functions._

val typeMapCol  = map(typeMap.flatMap{ case (k,v) => Seq(lit(k), lit(v)) }.toSeq : _*)
val intMapCol   = map(intensityMap.flatMap{ case (k,v) => Seq(lit(k), lit(v)) }.toSeq : _*)

val df_mapped = weatherData.withColumn(
    "Weather_Type",
    coalesce(
      typeMapCol(col("ValueForWindCharacter").cast("int")),
      lit("Clear_or_Other")
    )
  ).withColumn(
    "Weather_Intensity",
    coalesce(
      intMapCol(col("ValueForWindCharacter").cast("int")),
      lit("Not_Applicable")
    )
  )

typeMap: Map[Int,String] = Map(69 -> Rain_Snow_Mix, 5 -> Haze, 10 -> Fog_Mist, 56 -> Freezing_Precip, 42 -> Fog_Mist, 24 -> Freezing_Precip, 37 -> Blowing_Snow, 25 -> Shower, 52 -> Drizzle, 20 -> Drizzle, 57 -> Freezing_Precip, 29 -> Thunderstorm, 84 -> Shower, 61 -> Rain, 74 -> Snow, 60 -> Rain, 85 -> Shower, 28 -> Fog_Mist, 38 -> Blowing_Snow, 70 -> Snow, 21 -> Rain, 33 -> Dust_Sand, 65 -> Rain, 97 -> Thunderstorm, 53 -> Drizzle, 96 -> Thunderstorm, 41 -> Fog_Mist, 73 -> Snow, 32 -> Dust_Sand, 34 -> Dust_Sand, 45 -> Fog_Mist, 64 -> Rain, 17 -> Thunderstorm, 22 -> Snow, 44 -> Fog_Mist, 71 -> Snow, 54 -> Drizzle, 86 -> Shower, 81 -> Shower, 39 -> Blowing_Snow, 66 -> Freezing_Precip, 80 -> Shower, 35 -> Dust_Sand, 63 -> Rain, 18 -> Squall, 95 -> Thunderstorm, 50 -> Drizzle, 67 -> Freezin...


In [40]:
df_mapped
  .select(
    "ValueForWindCharacter",
    "Weather_Type",
    "Weather_Intensity"
  )
  .show(200, false)

+---------------------+---------------+-----------------+
|ValueForWindCharacter|Weather_Type   |Weather_Intensity|
+---------------------+---------------+-----------------+
|                     |Clear_or_Other |Not_Applicable   |
|                     |Clear_or_Other |Not_Applicable   |
|                     |Clear_or_Other |Not_Applicable   |
|                     |Clear_or_Other |Not_Applicable   |
|                     |Clear_or_Other |Not_Applicable   |
|                     |Clear_or_Other |Not_Applicable   |
|                     |Clear_or_Other |Not_Applicable   |
|                     |Clear_or_Other |Not_Applicable   |
|                     |Clear_or_Other |Not_Applicable   |
|                     |Clear_or_Other |Not_Applicable   |
|                     |Clear_or_Other |Not_Applicable   |
|                     |Clear_or_Other |Not_Applicable   |
|                     |Clear_or_Other |Not_Applicable   |
|                     |Clear_or_Other |Not_Applicable   |
|             

In [42]:
df_mapped
  .select(
    "ValueForWindCharacter",
    "Weather_Type",
    "Weather_Intensity"
  )
  .distinct()
  .orderBy("ValueForWindCharacter")
  .show(500, false)

+---------------------+---------------+-----------------+
|ValueForWindCharacter|Weather_Type   |Weather_Intensity|
+---------------------+---------------+-----------------+
|                     |Clear_or_Other |Not_Applicable   |
|100                  |Clear_or_Other |Not_Applicable   |
|101                  |Clear_or_Other |Not_Applicable   |
|103                  |Clear_or_Other |Not_Applicable   |
|104                  |Clear_or_Other |Not_Applicable   |
|105                  |Clear_or_Other |Not_Applicable   |
|106                  |Clear_or_Other |Not_Applicable   |
|107                  |Clear_or_Other |Not_Applicable   |
|108                  |Clear_or_Other |Not_Applicable   |
|109                  |Clear_or_Other |Not_Applicable   |
|11                   |Clear_or_Other |Not_Applicable   |
|111                  |Clear_or_Other |Not_Applicable   |
|112                  |Clear_or_Other |Not_Applicable   |
|114                  |Clear_or_Other |Not_Applicable   |
|115          